In [4]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [5]:

# define the path to the input images
input_path = r"C:\Users\user\Documents\GitHub\Hand Gesture\Hand-Gesture\input"

# define the path to the output preprocessed images
output_path = r"C:\Users\user\Documents\GitHub\Hand Gesture\Hand-Gesture\output"

# define the target size of the preprocessed images
target_size = (64, 64)

# define the preprocessing functions


def convert_to_gray(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray)
    return gray


def apply_gaussian_blur(img):
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    return blurred


def apply_threshold(img):
    _, thresh = cv2.threshold(
        img, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return thresh


def find_contours(img):
    contours, hierarchy = cv2.findContours(
        img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours, hierarchy


def find_largest_contour(contours):
    max_contour = max(contours, key=cv2.contourArea)
    return max_contour


def create_mask(img, contour):
    mask = cv2.drawContours(np.zeros_like(
        img), [contour], 0, (255, 255, 255), -1)
    return mask


def apply_mask(img, mask):
    masked_image = cv2.bitwise_and(img, img, mask=mask)
    return masked_image


def resize_image(img, size):
    resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return resized


def normalize_image(img):
    normalized = img.astype("float32") / 255.0
    return normalized


def get_hog_features(img):
    win_size = (64, 64)
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size,
                            block_stride, cell_size, nbins)
    hog_feats = hog.compute(img)
    return hog_feats


def get_sift_features(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return descriptors

def split_dataset(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def svm_model(X_train, y_train, X_test):
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred
    
def calc_accuracy(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [51]:
#preprocessing phase        
def preprocess(img_path):
    img = io.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blurred = apply_gaussian_blur(gray)
    blurred = np.uint8(blurred);
    thresh = apply_threshold(blurred)
    contours, hierarchy = find_contours(thresh)
    max_contour = find_largest_contour(contours)
    mask = create_mask(gray, max_contour)
    masked_image = apply_mask(img, mask)
    resized = resize_image(masked_image, target_size)
    normalized = normalize_image(resized)
    
    return normalized


In [53]:
# set the parameters for HoG
orientations = 9
pixels_per_cell = (50, 50)
cells_per_block = (3, 3)
visualize = False
transform_sqrt = False
normalize = True
load_features_from_file=False
save_feature=False
all_feats = []
all_imgs = []

image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

if load_features_from_file == False:
    for root, dirs, files in os.walk('dataset'):
        for dir in dirs:
            print(os.path.join(root, dir))
            for file in os.listdir(os.path.join(root, dir)):
                if any(file.lower().endswith(extension) for extension in image_extensions):
                    img_path = os.path.join(root, dir, file)
                    label = np.array([int(dir)])
                    
#                     normalized = preprocess(img_path)
                    normalized = io.imread(img_path, as_gray=True)
                    
                    #calculate the HoG features for the image
                    hog_result = hog(
                        normalized,
                        orientations = orientations,
                        pixels_per_cell = pixels_per_cell,
                        cells_per_block = cells_per_block,
                        visualize = visualize,
                        transform_sqrt = transform_sqrt,
                        block_norm = 'L2-Hys',
                        feature_vector = True
                    )

                    # unpack the HoG result into features and hog_image variables if necessary
                    if visualize:
                        features, hog_image = hog_result
                    else:
                        features = hog_result
                    
                    feats = np.concatenate((label, features.ravel()))
                    all_feats.append(feats)
           
else:
    all_feats = np.loadtxt("hog_features.txt")

features_array = np.array(all_feats, dtype=object)

if(save_feature):
    np.savetxt("hog_features.txt", features_array)


print("Feature Extraction Finished")

dataset/5
dataset/3
dataset/0
dataset/4
dataset/1
dataset/2
Feature Extraction Finished


In [ ]:
# get the optimal number of components to use

pca = PCA().fit(features_array[:, 1:])
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()

In [40]:
# apply PCA
pca = PCA(n_components=350)
Y = np.array(features_array[:, 0]).astype('int32')
X = pca.fit_transform(features_array[:, 1:])

# splitting data into train and test
X_train, X_test, y_train, y_test = split_dataset(X, Y)

print(X_train.shape)
print(y_train.shape)

# apply SVM
y_pred = svm_model(X_train, y_train, X_test)

print("SVM Finished")

(1456, 350)
(1456,)
SVM Finished


In [41]:
# calc accuracy
accuracy = calc_accuracy(y_test, y_pred)
print(accuracy)

0.7287671232876712
